<a href="https://colab.research.google.com/github/tubagokhan/RegNLPDataset/blob/main/SubTask2Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [15]:
import json
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define the mapping from numerical predictions to labels
label_mapping = ['contradiction', 'entailment', 'neutral']

# Load the pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-deberta-v3-large')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/nli-deberta-v3-large')

# Load the JSON data
with open('/content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM-Project/SharedTask/COBS_VER15.150823_answer_entailed_questions_subtask2_sample.json', 'r') as file:
    data = json.load(file)

item_count = 0  # Initialize item counter

# Iterate through each question-answer pair
for item in data:
    premise = item['Question']
    hypothesis = item['Answer']

    # Tokenize the premise and hypothesis
    features = tokenizer(premise, hypothesis, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    model.eval()
    with torch.no_grad():
        outputs = model(**features)
    print(outputs)
    # Get the predicted class and map it to the corresponding label
    prediction = torch.argmax(outputs.logits, dim=1).item()
    prediction_label = label_mapping[prediction]
    print(prediction_label)
    # Add the prediction label to the item
    item['Prediction'] = prediction_label

    item_count += 1  # Increment the counter

    print(f"Total items processed: {item_count}")

# Save the updated data back to a new JSON file
with open('/content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM-Project/SharedTask/COBS_VER15.150823_answer_entailed_questions_subtask2_sample_with_labels.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


SequenceClassifierOutput(loss=None, logits=tensor([[-4.7668, -0.5329,  4.1622]]), hidden_states=None, attentions=None)
neutral
Total items processed: 1
SequenceClassifierOutput(loss=None, logits=tensor([[-0.4749, -1.9692,  1.6881]]), hidden_states=None, attentions=None)
neutral
Total items processed: 2
SequenceClassifierOutput(loss=None, logits=tensor([[-3.5845, -3.0653,  4.9062]]), hidden_states=None, attentions=None)
neutral
Total items processed: 3
SequenceClassifierOutput(loss=None, logits=tensor([[-4.7853,  1.0738,  2.8910]]), hidden_states=None, attentions=None)
neutral
Total items processed: 4
SequenceClassifierOutput(loss=None, logits=tensor([[-3.4108, -2.3640,  4.3611]]), hidden_states=None, attentions=None)
neutral
Total items processed: 5
SequenceClassifierOutput(loss=None, logits=tensor([[-3.9342,  2.9345,  0.9419]]), hidden_states=None, attentions=None)
entailment
Total items processed: 6
SequenceClassifierOutput(loss=None, logits=tensor([[-5.3630,  2.3814,  2.4729]]), hid

In [ ]:
import json
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Function to calculate the mean entailment probability and return entailment probabilities
def calculate_mean_entailment_prob(Answer, Premise):
    # Tokenize the Answer text into sentences
    Hypothesis = sent_tokenize(Answer)

    # Load the NLI model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-deberta-v3-large')
    tokenizer = AutoTokenizer.from_pretrained('cross-encoder/nli-deberta-v3-large')

    # Prepare the premise-hypothesis pairs
    pairs = [(Premise, hypo) for hypo in Hypothesis]

    # Tokenize the pairs
    features = tokenizer(pairs, padding=True, truncation=True, return_tensors="pt")

    # Model evaluation
    model.eval()
    with torch.no_grad():
        logits = model(**features).logits
        probs = torch.softmax(logits, dim=1)
        entailment_probs = probs[:, 1].tolist()  # Extract the entailment probabilities

    # Calculate the mean entailment probability
    mean_entailment_prob = sum(entailment_probs) / len(entailment_probs) if entailment_probs else 0

    return mean_entailment_prob, entailment_probs

# Ensure nltk 'punkt' tokenizer data is available
nltk.download('punkt')

# Load the JSON data from the file
file_path = '/content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM-Project/SharedTask/COBS_VER15.150823_answer_entailed_questions_subtask2_sample_with_labels.json'

# Read the JSON data
with open(file_path, 'r') as file:
    data = json.load(file)

# Initialize a counter
item_count = 0

# Process each item in the JSON data
for item in data:
    Answer = item["Answer"]
    Premise = item["Question"]

    # Calculate mean entailment probability and entailment probabilities for each pair
    mean_entailment_prob, entailment_probs = calculate_mean_entailment_prob(Answer, Premise)

    # Add the results back to the item in the JSON data
    item["Mean_Entailment_Prob"] = mean_entailment_prob
    item["Entailment_Probs"] = entailment_probs

    # Increment the counter
    item_count += 1

    # Print the total number of items processed
    print(f"Total number of items processed: {item_count}")

# Save the updated JSON data back to a new file
updated_file_path = '/content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM-Project/SharedTask/COBS_VER15.150823_answer_entailed_questions_subtask2_sample_with_labels.json'
with open(updated_file_path, 'w') as file:
    json.dump(data, file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total number of items processed: 1
Total number of items processed: 2
Total number of items processed: 3
Total number of items processed: 4
Total number of items processed: 5
Total number of items processed: 6
Total number of items processed: 7
Total number of items processed: 8
Total number of items processed: 9
Total number of items processed: 10
Total number of items processed: 11
Total number of items processed: 12
Total number of items processed: 13
Total number of items processed: 14
Total number of items processed: 15
Total number of items processed: 16
Total number of items processed: 17
Total number of items processed: 18
Total number of items processed: 19
Total number of items processed: 20
Total number of items processed: 21
Total number of items processed: 22
Total number of items processed: 23
Total number of items processed: 24
Total number of items processed: 25
Total number of items processed: 26
Total number of items processed: 27
Total number of items processed: 28
T